**Partie IV** 

Dans cette partie, on s'intéresse à deux autres aspects importants du deep learning : l'accélération de l'apprentissage avec les cartes GPU et la possibilité d'utiliser des réseaux préentraînés.

Pour illustrer le premier aspect, nous utiliserons les cartes gpu mises à disposition sous google colab. 
Pour ce faire, avant de commencer la lecture du notebook, aller à **Modifier**/**Modifier les param du notebook** et sélectionner un gpu. 

Pour le second aspect, nous travaillerons sur un problème de classification binaire à partir d'un jeu de données de taille réduite (hymenoptera). Nous voyons l'intérêt d'utiliser un réseau qui a déjà été entraîné sur un jeu plus grand et sur une tâche de classification plus générale. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import torch
import torchvision
import torch.nn as nn   
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#vérification de la mise à disponibilité de la carte gpu:
print(torch.cuda.get_device_name(0))

Tesla T4


**A.** Visualisation du jeu hymenoptera:

In [ ]:
data_dir = '/content/drive/MyDrive/TP_ENM/data/hymenoptera_data'

print(os.listdir(data_dir))

Le jeu de données se présente sous une forme standard, on va pouvoir le manipuler avec un objet dataset prêt à l'emploi de la classe *datasets.ImageFolder*.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

print('taille du jeu de données:' )
print(dataset_sizes)

Comme le jeu de donnée mis à disposition est de très petite taille, il faut l'exploiter au maximum. On va donc produire de nouvelles images par des transformations supplémentaires qui conservent la nature de l'objet (augmentation de données). Dans le code, *transforms.RandomResizedCrop()* et *transforms.RandomHorizontalFlip()* et  *transforms.RandomVerticalFlip()* appliquent avec une probabilité de 1/2 une symétrie d'axe horizontal ou vertical. \\
Remarquez que ces transformations n'auraient pas pu être exploitées avec d'autres jeux de données comme MNIST: le symétrique d'une image de chiffre n'est généralement pas une image de chiffre. Il faut donc souvent adapter la transformation à la nature du jeu de données.
En dessous, on présente quelques images.



In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

class_names = image_datasets['train'].classes

out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

**B.** Utilisation d'une carte graphique \\

Dans un premier temps, nous allons vérifier que pour des architectures classiques comme le ResNet, l'utilisation d'une carte gpu améliore sensiblement le temps de calcul. 
Commençons par charger la plus légère des architectures de type ResNet et écrivons la procédure d'apprentissage dans une fonction, une bonne fois pour toutes.

**Exercice** 

- Charger un ResNet18 non pré-entraîné. Combien contient-il de poids au total ? Vérifier [ici](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html).

- Combien de neurones la dernière couche du réseau comporte-t-elle ?

- Compléter la fonction *train_model* prenant pour arguments un model, une fonction de coût, un optimizer et un nombre d'époques.


In [ ]:
model = models.resnet18( ... )   
print(model)

In [ ]:
nb_weights = 0

...

print(nb_weights)

In [ ]:
# Nb de neurones dans la dernière couche : 

In [ ]:
def train_model(model, loss_ft, optimizer, num_epochs=1):           

    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            # initialisation des compteurs

            for inputs, labels in dataloaders[phase]:

                # mise à zéro des incréments dans l'optimizer

                # calcul des sorties (proba) et des prédictions (classe)

                # fonction de coût

                # gradients et backprop si entraînement 

                # compteurs

            # Calcul du coût moyen et de la justesse sur l'époque

    return model


Pour entraîner un resnet18, il faut d'abord modifier la dernière couche du classifieur de façon à ce qu'il y ait autant de neurones que de classes:

In [ ]:
#Taille du vecteur d'entrée
num_ftrs = model.fc.in_features    
print(num_ftrs)

#Nouvelle couche à deux neurones 
model.fc = nn.Linear(num_ftrs, 2)  

Pour le calcul de la log vraisemblance, il faudrait aussi ajouter une couche LogSoftmax. En effet, le ResNet comme les autres modèles standard s'arrête à la partie linéaire d'une couche complètement connectée.
Le plus simple est d'utiliser une fonction de coût qui incorpore la LogSoftmax. \\
Sous pytorch, c'est *nn.CrossEntropyLoss* qui combine *LogSoftmax* et *NLLLoss*.

In [ ]:
loss_ft =  nn.CrossEntropyLoss()

Lançons maintenant un entraînement sur une époque par batches de 64 images:




In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
              
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)    

model = train_model(model, loss_ft, optimizer, num_epochs=1)

Avec plus de 100 M de poids, l'entraînement d'un ResNet sur CPU est beaucoup plus long que les réseaux de la partie III. \\
Reprenons le même entraînement en utilisant la carte graphique. Pour cela, on doit préciser la carte sur laquelle passer les tenseurs: 

In [ ]:
device = torch.device("cuda:0") # if torch.cuda.is_available() else "cpu")

Ensuite, on passe le model sur la carte graphique:

In [ ]:
model = model.to(device)

Pour passer un tenseur sur la carte, la syntaxe est la même:

In [ ]:
x = torch.rand(2,1,4,4)
print(x)
x = x.to(device)
print(x)

**remarques :**

- Le nom de la carte apparaît maintenant lorsqu'on affiche le tenseur.
- On utilise aussi la méthode .cuda() lorsqu'il n'y a qu'une seule carte GPU. 
- Pour rapatrier un tenseur (ou un modèle), on utilise la méthode .cpu() : 

In [ ]:
print(x.cpu())

**Exercice :** A partir de la fonction *train_model*, définir une fonction train_model_gpu où le calcul de l'output et des gradients se font sur la carte gpu. Comparer les performances en temps.

In [ ]:
def train_model_gpu(model, loss_ft, optimizer, num_epochs=1) :
    ...

In [ ]:
model = train_model_gpu(model, loss_ft, optimizer, num_epochs=1)

**C.** Amélioration de la procédure d'entraînement

Avant de comparer différentes approches de transfert, nous allons un peu améliorer la procédure d'entraînement.
D'abord, on peut ajouter une inertie (momentum) à la descente de gradient. Le  [calcul de l'incrément](https://pytorch.org/docs/master/optim.html#torch.optim.SGD) dépendra non seulement du gradient courant mais aussi des valeurs d'incrément passées, contenues dans $d_i$: \\

\begin{equation}
d_i^{t+1} = momentum \times d_i^{t} +    \dfrac{\partial \mathcal{L_{batch}^{t+1}}} {\partial{\omega_i}} \\
w_i^{t+1}  = w_i^{t} - lr \times d_i^{t+1}
\end{equation}


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Ensuite nous ajoutons un *scheduler* qui permet de diminuer progressivement le taux d'apprentissage *lr*. Avec le *scheduler* suivant, toutes les cinq époques, *lr* est multiplié par gamma = 0.1. \\
Le *scheduler* doit avoir accès au *lr* contenu dans *optimizer*. Ce dernier est donc passé comme argument. Le *scheduler* n'agit sur le *lr* qu'à la fin d'une époque. Pour que cette action soit effective, il faudra ajouter en dehors de la boucle d'itération du loader: \\

*scheduler.step()* 


In [ ]:
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

**Exercice**: intégrer ces éléments dans la fonction d'entraînement. Faire en sorte, aussi, que la fonction d'entraînement renvoie les listes des scores (loss et justesse). Vérifier que le taux d'apprentissage décroit comme prévu.  

*N.B.:* Sous pytorch, chaque poids est associé à un taux d'apprentissage.
        Ces taux d'apprentissage sont stockés dans *optimizer.param_groups*.

**Remarque:** On utilise généralement le paramétrage qui a obtenu les meilleurs scores sur le jeu de validation. Pour ça, on enregistre les poids en cours d'apprentissage, dès qu'un record en validation est atteint. Noter qu'en sélectionnant ainsi le modèle, on apprend le jeu de validation. \\
C'est la raison pour laquelle l'évaluation d'un modèle sélectionné sur le jeu de validation a toujours lieu sur un **jeu de test** indépendant des jeux de validation et d'entraînement.

Pour information, voilà comment garder en mémoire les poids d'un modèle sous pytorch:

In [ ]:
import copy

# Dans la boucle :
if phase == "val" and epoch_acc > best_val_acc :
  best_model_wts = copy.deepcopy(model.state_dict())
  best_val_acc = epoch_acc

# Après apprentissage :
model.load_state_dict(best_model_wts)

**D.** Effet d'un pré-entraînement sur les performances

L'entraînement est plus rapide sur GPU, mais il ne conduit qu'à un score très médiocre, à peine meilleur que le hasard.
Pour améliorer les performances, une idée simple consiste utiliser un réseau entraîné sur une tâche similaire (ou plus générale) comme point de départ de l'apprentissage. On parle de ré-entraînement (ou **fine tuning**). 
Ici, cela va particulièrement bien marcher avec des réseaux entraînés sur ImageNet, dont les les filtres de convolution sont déjà très riches. 

**Exercice**: 
Modifier la fonction d'entraînement de façon à récupérer les justesses successives.
Reprendre l'apprentissage avec un ResNet18 non-préentraîné sur 25 époques. Comparer avec le même réseau préentraîné sur ImageNet (utiliser les courbes d'apprentissage). 

In [ ]:
# Nouvelle fonction d'entraînement:

def train_model_gpu(model, loss_ft, optimizer, num_epochs=1):
    ...
    return model, accs


In [ ]:
# Apprentissage "from scratch" (poids initialisés au hasard) :
# définition des modèle, optimizer, scheduler
# et lancement de l'entraînement

resnet_scratch = torchvision.models.resnet18(pretrained=False)

...

resnet_scratch, accs_scratch = train_model_gpu(resnet_scratch, loss_function, optimizer, num_epochs=1)

In [ ]:
# fine tuning :
# définition des modèle, optimizer, scheduler
# et lancement de l'entraînement


resnet_ft = torchvision.models.resnet18(pretrained=True)

...

resnet_ft, , accs_ft = train_model_gpu(resnet_ft, loss_function, optimizer, num_epochs=1)

In [ ]:
# Comparaison des courbes d'apprentissage (justesse seulement)

L'approche par fine tuning connaît de nombreuses variantes qui s'intègrent dans le cadre plus général de **l'apprentissage par transfert** (transfert learning).
Le fine tuning partiel qui est illustré dans l'exercice suivant est l'une de ces variantes.

**Exercice :** Au lieu de réapprendre tous les poids, on peut se contenter de ceux du classifieur. On dit qu'on "gèle" les autres poids lors du ré-entraînement (**freezing**).
Mettre en place cette approche et comparer avec les précédentes.

In [ ]:
# freezing :
# parcourir les paramètres du modèle 
# et préciser ceux dont on ne veut
# pas calculer les gradient (méthode .requires_grad)


resnet_freezing = torchvision.models.resnet18(pretrained=True)

...

for param in ...
    param.requires_grad = False

In [ ]:
# Définition des modèle, optimizer, scheduler
# et lancement de l'entraînement

In [ ]:
# Comparaison

Au final, pour ce petit jeu de données, le réapprentissage de la dernière couche permet de faire aussi bien qu'un entraînement global. \\
Voyons pour terminer quelques prédictions du modèle sur le jeu de validation:

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


In [ ]:
visualize_model(resnet_freezing)